In [60]:
import os
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

# Load data

In [61]:

# Loading the data
df = pd.read_csv("../data/fraudTrain.csv")
# df = pd.read_csv("../data/fraudTest.csv")

# Drop the column named 'Unnamed: 0' (unnecessary index column)
df = df.drop(columns=['Unnamed: 0'])

# Convert date/time columns
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], errors='coerce')
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

# Keep IDs as string/object
df['cc_num'] = df['cc_num'].astype(str)
df['trans_num'] = df['trans_num'].astype(str)

# Convert categorical/text columns
categorical_cols = ['merchant', 'category', 'first', 'last', 'gender', 
                    'street', 'city', 'state', 'zip', 'job']
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Convert to Unix timestamp (in seconds)
df['unix_trans_time'] = df['trans_date_trans_time'].astype('int64') // 10**9
df['age'] = (df['trans_date_trans_time'] - df['dob']).dt.days / 365.25 # account for leap years

# Compute number of distinct categories per merchant
merchant_category_counts = df.groupby("merchant")["category"].transform("nunique")
# Add it as a new column
df["nb_categories"] = merchant_category_counts

print(df.dtypes)

trans_date_trans_time    datetime64[ns]
cc_num                           object
merchant                       category
category                       category
amt                             float64
first                          category
last                           category
gender                         category
street                         category
city                           category
state                          category
zip                            category
lat                             float64
long                            float64
city_pop                          int64
job                            category
dob                      datetime64[ns]
trans_num                        object
unix_time                         int64
merch_lat                       float64
merch_long                      float64
is_fraud                          int64
unix_trans_time                   int64
age                             float64
nb_categories                     int64


/tmp/ipykernel_21934/6895694.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merchant_category_counts = df.groupby("merchant")["category"].transform("nunique")


# Create node ID mappings

In [62]:
# Create numeric IDs for graph nodes (cards, merchants, transactions)

# treat each unique card number as a category
card_ids = df["cc_num"].astype("category").cat.codes
# Add a new column card_id
df["card_id"] = card_ids

# treat each unique merchant as a category
merchant_ids = df["merchant"].astype("category").cat.codes
# Add a new column merchant_id
df["merchant_id"] = merchant_ids

# Each row is one transaction
df["transaction_id"] = range(len(df))


In [63]:
# Number of transaction nodes
print("Number of transaction : ", len(df))

# Count how many unique cards
print("Number of cards : ", card_ids.nunique())

# Count how many unique merchant
print("Number of merchants : ", merchant_ids.nunique())

Number of transaction :  1296675
Number of cards :  983
Number of merchants :  693


# Build node features

In [64]:
# Fenêtre temporelle (en secondes) utilisée comme valeur par défaut
# lorsque aucune transaction précédente n’existe
FEATURE_WINDOW = 3600  # 1 heure

# Encodage des variables catégorielles
df["category_idx"] = df["category"].astype("category").cat.codes
df["gender_idx"] = df["gender"].astype("category").cat.codes
df["job_idx"] = df["job"].astype("category").cat.codes

# Features temporelles
# Ces variables capturent les comportements cycliques et
# les habitudes de consommation
df["hour"] = df["trans_date_trans_time"].dt.hour
df["dayofweek"] = df["trans_date_trans_time"].dt.dayofweek
df["is_weekend"] = df["dayofweek"].isin([5, 6]).astype(int)


# Dynamique temporelle de la carte
# Tri chronologique des transactions par carte
df = df.sort_values(["card_id", "unix_trans_time"])
# Temps écoulé (en secondes) depuis la transaction précédente
# pour la même carte
# Un intervalle très court peut indiquer une activité anormale
df["card_time_since_prev_tx"] = (
    df.groupby("card_id")["unix_trans_time"]
    .diff()
    .fillna(FEATURE_WINDOW)
)


# Déviation du montant par rapport à l’historique de la carte
# Montant moyen historique de la carte (jusqu’à t-1)
df["card_amt_mean"] = (
    df.groupby("card_id")["amt"]
    .expanding()
    .mean()
    .shift()
    .reset_index(level=0, drop=True)
)
# Écart-type historique des montants de la carte (jusqu’à t-1)
df["card_amt_std"] = (
    df.groupby("card_id")["amt"]
    .expanding()
    .std()
    .shift()
    .reset_index(level=0, drop=True)
)
# Remplacement des valeurs manquantes (premières transactions)
df[["card_amt_mean", "card_amt_std"]] = df[["card_amt_mean", "card_amt_std"]].fillna(0)

# Z-score : mesure à quel point le montant est atypique pour cette carte
df["amt_zscore"] = (
    (df["amt"] - df["card_amt_mean"]) /
    (df["card_amt_std"] + 1e-6)
)


# Distance géographique entre transactions consécutives
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371.0  # rayon Terre en km

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2

    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))


# Tri chronologique des transactions par carte
df = df.sort_values(["card_id", "unix_trans_time"])
# Coordonnées du marchand de la transaction précédente
df["prev_merch_lat"] = df.groupby("card_id")["merch_lat"].shift()
df["prev_merch_long"] = df.groupby("card_id")["merch_long"].shift()
# Distance géographique entre deux transactions consécutives
# Une grande distance sur un temps court est un fort signal de fraude
df["geo_dist"] = haversine_np(
    df["merch_lat"], df["merch_long"],
    df["prev_merch_lat"], df["prev_merch_long"]
)
# Valeur nulle pour la première transaction
df["geo_dist"] = df["geo_dist"].fillna(0)

# Nouveau merchant pour la carte
# Indique si ce marchand n’a jamais été utilisé auparavant
# par cette carte (1 = nouveau marchand)
df["is_new_merchant"] = (
    df.groupby("card_id")["merchant"]
    .transform(lambda x: ~x.duplicated())
    .astype(int)
)

# Dynamique temporelle du merchant
# Temps écoulé depuis la dernière transaction chez ce marchand
# Un afflux soudain de transactions peut être suspect
df["merchant_time_since_prev_tx"] = (
    df.groupby("merchant_id")["unix_trans_time"]
    .diff()
    .fillna(FEATURE_WINDOW)
)

# Montant moyen historique des transactions du merchant
# (calculé uniquement sur le passé)
df["merchant_avg_amt"] = (
    df.groupby("merchant_id")["amt"]
    .expanding()
    .mean()
    .shift()
    .reset_index(level=0, drop=True)
).fillna(0)



# Create PyG graphs

In [65]:
def create_graph(df, EDGE_WINDOW = 3600 * 2):
    """
    Crée un graphe PyG à partir d'un dataframe df.
    """
    # Node features
    node_features = torch.tensor(
        df[[
            # Transaction features
            "amt",
            "card_time_since_prev_tx",
            "hour",
            "dayofweek",
            "is_weekend",
            "age",
            "is_new_merchant",
            "geo_dist",
            "merch_lat", "merch_long",
            "category_idx",

            # Card features
            "amt_zscore",
            "card_amt_mean",
            "card_amt_std",
            "gender_idx", "job_idx",
            "lat", "long", "city_pop",

            # Mercahnt feature
            "merchant_avg_amt",
            "merchant_time_since_prev_tx"
            
        ]].values,
        dtype=torch.float
    )

    node_labels = torch.tensor(
        df["is_fraud"].values,
        dtype=torch.long
    )


    # Mapping transaction_id -> index PyG
    tx2idx = {tx: i for i, tx in enumerate(df["transaction_id"].values)}

    # Create edges
    edges = []
    edge_attrs = []

    # Création des arêtes pour transactions de la même carte
    # On regroupe les transactions par carte (card_id)
    # puis on relie les transactions consécutives dans la fenêtre EDGE_WINDOW
    for _, group in df.groupby("card_id"):
        # Tri chronologique des transactions
        group = group.sort_values("unix_trans_time")
        tx = group["transaction_id"].values
        t = group["unix_trans_time"].values

        # Parcours des transactions consécutives
        for i in range(len(tx) - 1):
            # Si deux transactions sont assez proches dans le temps
            if t[i+1] - t[i] <= EDGE_WINDOW:
                # On ajoute une arête bidirectionnelle
                edges.append([tx2idx[tx[i]], tx2idx[tx[i+1]]])
                edges.append([tx2idx[tx[i+1]], tx2idx[tx[i]]])

                # Attribut de l'arête : [same_card, same_merchant]
                # Ici, same_card = 1, same_merchant = 
                edge_attrs.append([1, 0])
                edge_attrs.append([1, 0])


    # Création des arêtes pour transactions du même merchant
    # Même logique que pour les cartes
    # On relie les transactions consécutives chez le même marchand
    for _, group in df.groupby("merchant_id"):
        # Tri chronologique des transactions
        group = group.sort_values("unix_trans_time")
        tx = group["transaction_id"].values
        t = group["unix_trans_time"].values

        for i in range(len(tx) - 1):
            # Si deux transactions sont assez proches dans le temps
            if t[i+1] - t[i] <= EDGE_WINDOW:
                # Arêtes bidirectionnelles
                edges.append([tx2idx[tx[i]], tx2idx[tx[i+1]]])
                edges.append([tx2idx[tx[i+1]], tx2idx[tx[i]]])

                # Attribut de l'arête : [same_card, same_merchant]
                # Ici, same_card = 0, same_merchant = 1
                edge_attrs.append([0, 1])
                edge_attrs.append([0, 1])

    
    # Assemble Data
    # Conversion des arêtes et attributs en tenseurs PyTorch
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)

    # Création du graphe PyTorch Geometric
    data = Data(
        x=node_features,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=node_labels
    )

    data.num_nodes = node_features.size(0)
    
    return data


In [66]:
# durée de chaque sous-graphe (batch) en mois
nb_months = 1

# Découpage par nb_months
graphs = []
df["batch_index"] = df["unix_trans_time"].apply(lambda x: (pd.to_datetime(x, unit='s').month - 1)//nb_months + 1)
for period, period_df in df.groupby("batch_index"):
    graph = create_graph(period_df.reset_index(drop=True))
    graphs.append(graph)
    print(f"--- Batch {period} ---")
    print(graph)
    print("Nombre de noeuds:", graph.num_nodes)
    print("Nombre d'arêtes:", graph.num_edges)
    print("Dimension features noeuds:", graph.x.shape[1])
    print("Dimension features arêtes:", graph.edge_attr.shape[1])
    print()

print(f"{len(graphs)} graphes créés, un pour chaque période de {nb_months} mois.")

--- Batch 1 ---
Data(x=[104727, 21], edge_index=[2, 106004], edge_attr=[106004, 2], y=[104727], num_nodes=104727)
Nombre de noeuds: 104727
Nombre d'arêtes: 106004
Dimension features noeuds: 21
Dimension features arêtes: 2

--- Batch 2 ---
Data(x=[97657, 21], edge_index=[2, 101426], edge_attr=[101426, 2], y=[97657], num_nodes=97657)
Nombre de noeuds: 97657
Nombre d'arêtes: 101426
Dimension features noeuds: 21
Dimension features arêtes: 2

--- Batch 3 ---
Data(x=[143789, 21], edge_index=[2, 185850], edge_attr=[185850, 2], y=[143789], num_nodes=143789)
Nombre de noeuds: 143789
Nombre d'arêtes: 185850
Dimension features noeuds: 21
Dimension features arêtes: 2

--- Batch 4 ---
Data(x=[134970, 21], edge_index=[2, 171244], edge_attr=[171244, 2], y=[134970], num_nodes=134970)
Nombre de noeuds: 134970
Nombre d'arêtes: 171244
Dimension features noeuds: 21
Dimension features arêtes: 2

--- Batch 5 ---
Data(x=[146875, 21], edge_index=[2, 193668], edge_attr=[193668, 2], y=[146875], num_nodes=146875

In [67]:
# Save to a file
# Make sure the folder exists
save_dir = "graphs/train"
# save_dir = "graphs/test"
os.makedirs(save_dir, exist_ok=True)
for i, graph in enumerate(graphs, start=1):
    file_path = os.path.join(save_dir, f"graph_batch_{i}.pt")
    torch.save(graph, file_path)
    print(f"Graphe batch {i} enregistré : {file_path}")


Graphe batch 1 enregistré : graphs/train/graph_batch_1.pt
Graphe batch 2 enregistré : graphs/train/graph_batch_2.pt
Graphe batch 3 enregistré : graphs/train/graph_batch_3.pt
Graphe batch 4 enregistré : graphs/train/graph_batch_4.pt
Graphe batch 5 enregistré : graphs/train/graph_batch_5.pt
Graphe batch 6 enregistré : graphs/train/graph_batch_6.pt
Graphe batch 7 enregistré : graphs/train/graph_batch_7.pt
Graphe batch 8 enregistré : graphs/train/graph_batch_8.pt
Graphe batch 9 enregistré : graphs/train/graph_batch_9.pt
Graphe batch 10 enregistré : graphs/train/graph_batch_10.pt
Graphe batch 11 enregistré : graphs/train/graph_batch_11.pt
Graphe batch 12 enregistré : graphs/train/graph_batch_12.pt
